### This notebook clusters teh consumer spending to cluster states by similar profiles

In [105]:
# Required imports
import os
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")

## Load and read foodprice.csv to get desired columns

In [106]:
cpiDf = pd.read_csv(
    Path("resources/foodPrice.csv")
)
cpiDf.head()

,Cereals-and-bakery-products,"Meats,-poultry,-fish,-and-eggs",Dairy-and-related-products,Fruits-and-vegetables,Other-food-at-home,Date
0,79.7,90.4,87.3,76.5,83.0,1980-01-01
1,80.6,89.7,87.8,76.0,84.3,1980-02-01
2,81.2,90.3,88.1,77.4,85.5,1980-03-01
3,82.4,89.3,88.9,80.2,86.4,1980-04-01
4,83.2,88.0,90.5,82.1,87.3,1980-05-01


In [107]:
cpiDf.rename({'Dairy-and-related-products': 'Dairy-products'},axis='columns',inplace = True)
cats = cpiDf.columns.to_list()
cats = [i.replace('-',' ') for i in cats if i != 'Date']

## Reshape data into shape for easier clustering

In [108]:
spendDf = pd.read_csv(Path(f"resources/spending_2005-2022.csv"),index_col = 'Item')
spendDf = spendDf.loc[cats]
yearAndCat = []
for cat,year in zip(spendDf.index,spendDf['Year']):
    ind = f'{cat} ({int(year)})'
    yearAndCat.append(ind)
spendDf.index = yearAndCat
spendDf.drop(columns='Year',inplace=True)
spendDf = spendDf.T
spendDf = spendDf.reset_index().rename({'index':'City'},axis='columns')

spendDf = spendDf.dropna(axis='index').reset_index(drop=True)
spendDf.head()

,City,Cereals and bakery products (2005),Cereals and bakery products (2006),Cereals and bakery products (2007),Cereals and bakery products (2008),Cereals and bakery products (2009),Cereals and bakery products (2010),Cereals and bakery products (2011),Cereals and bakery products (2012),Cereals and bakery products (2013),...,Other food at home (2013),Other food at home (2014),Other food at home (2015),Other food at home (2016),Other food at home (2017),Other food at home (2018),Other food at home (2019),Other food at home (2020),Other food at home (2021),Other food at home (2022)
0,Chicago,473.0,491.0,537.0,561.0,584.0,615.0,602.0,596.0,664.0,...,1598.0,1652.0,1578.0,1562.0,1595.0,1666.0,1777.0,2119.0,2174.0,2035.0
1,Detroit,545.0,527.0,579.0,628.0,540.0,546.0,573.0,583.0,618.0,...,1748.0,1691.0,1400.0,1422.0,1463.0,1626.0,1636.0,1762.0,1887.0,2051.0
2,Minneapolis-St.Paul,493.0,470.0,517.0,536.0,518.0,582.0,632.0,639.0,582.0,...,1608.0,1764.0,1792.0,1682.0,1743.0,1833.0,2126.0,2152.0,2001.0,2317.0
3,New York,568.0,533.0,528.0,535.0,558.0,565.0,597.0,601.0,590.0,...,1236.0,1283.0,1228.0,1191.0,1364.0,1500.0,1635.0,1807.0,1994.0,2163.0
4,Philadelphia,522.0,454.0,408.0,467.0,526.0,523.0,529.0,605.0,656.0,...,1433.0,1365.0,1305.0,1444.0,1637.0,1740.0,1701.0,1968.0,2230.0,2252.0


In [109]:
## maybe calculate average changes
fullAvg = []
for cat in cats:
    temp = spendDf[[i for i in spendDf.columns if cat in i]]
    catAvgChange = np.mean(np.diff(temp)/np.array(temp)[:,:17],axis=1)
    spendDf[f'{cat} Average Change'] = catAvgChange*100
    fullAvg.append(catAvgChange)
spendDf['Average Change'] = np.mean(fullAvg,axis=0)*100

## Plot cities and two dimensions, we see that there aren't clear groups

In [110]:
spendDf.hvplot.scatter(
    x="Meats, poultry, fish, and eggs (2022)",
    y="Fruits and vegetables (2022)",
    s='Average Change',
    scale=5,
    by="City")

:NdOverlay   [City]
   :Scatter   [Meats, poultry, fish, and eggs (2022)]   (Fruits and vegetables (2022),Average Change)

In [111]:
## binarize city category
spendDfDummy = pd.get_dummies(spendDf)
spendDfDummy.head()

,Cereals and bakery products (2005),Cereals and bakery products (2006),Cereals and bakery products (2007),Cereals and bakery products (2008),Cereals and bakery products (2009),Cereals and bakery products (2010),Cereals and bakery products (2011),Cereals and bakery products (2012),Cereals and bakery products (2013),Cereals and bakery products (2014),...,City_Los Angeles,City_Miami,City_Minneapolis-St.Paul,City_New York,City_Philadelphia,City_Phoenix,City_San Diego,City_San Francisco,City_Seattle,"City_Washington, D.C."
0,473.0,491.0,537.0,561.0,584.0,615.0,602.0,596.0,664.0,661.0,...,0,0,0,0,0,0,0,0,0,0
1,545.0,527.0,579.0,628.0,540.0,546.0,573.0,583.0,618.0,544.0,...,0,0,0,0,0,0,0,0,0,0
2,493.0,470.0,517.0,536.0,518.0,582.0,632.0,639.0,582.0,525.0,...,0,0,1,0,0,0,0,0,0,0
3,568.0,533.0,528.0,535.0,558.0,565.0,597.0,601.0,590.0,609.0,...,0,0,0,1,0,0,0,0,0,0
4,522.0,454.0,408.0,467.0,526.0,523.0,529.0,605.0,656.0,613.0,...,0,0,0,0,1,0,0,0,0,0


## Attempt to cluster data, see some clusters but still not entirely clear

In [112]:
model = KMeans(n_clusters=3, random_state=1)
model.fit(spendDfDummy)
clusters = model.predict(spendDfDummy)
print(clusters)

[0 1 0 2 2 0 2 2 2 1 1 2 0 0 0 1]


In [113]:
clusterDf = spendDf.copy()
clusterDf['Cluster'] = clusters
clusterDf.hvplot.scatter(
    x="Meats, poultry, fish, and eggs (2022)",
    y="Fruits and vegetables (2022)",
    by="Cluster",
    hover_cols=['City'])

:NdOverlay   [Cluster]
   :Scatter   [Meats, poultry, fish, and eggs (2022)]   (Fruits and vegetables (2022),City)

In [114]:
## Initialize PCA and use elbow to find beswt clustering value

In [115]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
spendPca = pca.fit_transform(spendDfDummy)
pca.explained_variance_ratio_

array([0.4732204 , 0.21885699, 0.10835317])

In [116]:
# Create the PCA DataFrame
spendPcaDf = pd.DataFrame(
    spendPca,
    columns=["PCA1", "PCA2",'PCA3']
)

# Review the PCA DataFrame
spendPcaDf.head()

,PCA1,PCA2,PCA3
0,322.766633,253.550649,-388.691435
1,-302.439338,221.621004,-391.621210
2,413.076418,761.781164,20.110776
3,369.436832,-1007.285501,-353.576808
4,148.979148,-138.094588,386.433540


In [117]:
# Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

# Append the value of the computed inertia from the `inertia_` attribute of teh KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(spendPcaDf)
    inertia.append(k_model.inertia_)

# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,2.572690e+07
1,2,1.550918e+07
2,3,1.080212e+07
3,4,8.462482e+06
4,5,6.301915e+06


In [118]:
# Plot the Elbow Curve
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

## Use PCA values to cluster data

In [119]:
# Define the model with 3 clusters
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(spendPcaDf)

# Make predictions
k_3 = model.predict(spendPcaDf)

# Create a copy of the PCA DataFrame
spendPcaDf_pred = spendPcaDf.copy()

# Add a class column with the labels
spendPcaDf_pred["Clusters"] = k_3

# spendPcaDf_pred['City'] = spendDf['City']
spendPcaDf_pred = pd.merge(spendPcaDf_pred,spendDf,left_index=True,right_index=True)


In [120]:
spendPcaDf_pred.head()

,PCA1,PCA2,PCA3,Clusters,City,Cereals and bakery products (2005),Cereals and bakery products (2006),Cereals and bakery products (2007),Cereals and bakery products (2008),Cereals and bakery products (2009),...,Other food at home (2019),Other food at home (2020),Other food at home (2021),Other food at home (2022),Cereals and bakery products Average Change,"Meats, poultry, fish, and eggs Average Change",Dairy products Average Change,Fruits and vegetables Average Change,Other food at home Average Change,Average Change
0,322.766633,253.550649,-388.691435,0,Chicago,473.0,491.0,537.0,561.0,584.0,...,1777.0,2119.0,2174.0,2035.0,2.690843,2.802966,1.862723,3.743654,3.391493,2.898336
1,-302.439338,221.621004,-391.621210,2,Detroit,545.0,527.0,579.0,628.0,540.0,...,1636.0,1762.0,1887.0,2051.0,1.691562,0.940985,1.496953,3.113009,2.876458,2.023793
2,413.076418,761.781164,20.110776,0,Minneapolis-St.Paul,493.0,470.0,517.0,536.0,518.0,...,2126.0,2152.0,2001.0,2317.0,2.093101,3.503035,1.546455,4.831423,3.661225,3.127048
3,369.436832,-1007.285501,-353.576808,2,New York,568.0,533.0,528.0,535.0,558.0,...,1635.0,1807.0,1994.0,2163.0,2.324064,2.194653,2.343244,4.041798,4.141036,3.008959
4,148.979148,-138.094588,386.433540,2,Philadelphia,522.0,454.0,408.0,467.0,526.0,...,1701.0,1968.0,2230.0,2252.0,3.033625,2.189416,2.191024,4.704111,4.513988,3.326433


## Now we see clear clusters, we can imply that these cities have similar consumer spending profiles through the years

In [121]:
spendPcaDf_pred.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    s = 'Average Change',
    scale = 5,
    by="Clusters",
    hover_cols=['City']

)


:NdOverlay   [Clusters]
   :Scatter   [PCA1]   (PCA2,Average Change,City)

In [122]:
import plotly.express as px

pcaPriceDfPredictionsPlot = spendPcaDf_pred.reset_index()
# fig = px.scatter_3d(spendPcaDf_pred, x="Cereals and bakery products Average Change", y="Fruits and vegetables Average Change", z="Meats, poultry, fish, and eggs Average Change",size = 'Average Change',
#                     color='customer_segments',color_continuous_scale= px.colors.sequential.Rainbow,hover_name='City')
fig = px.scatter_3d(spendPcaDf_pred, x="PCA1", y="PCA2", z="PCA3",size = 'Average Change',
                    color='Clusters',color_continuous_scale= px.colors.sequential.Rainbow,hover_name='City')

fig.show()

## Exploration of data (Ignore)

In [123]:
cpiDf = pd.read_csv(
    Path("resources/foodPrice.csv")
)

cpiDf.head()
cpiDf.rename({'Dairy-and-related-products': 'Dairy-products'},axis='columns',inplace = True)
cats = cpiDf.columns.to_list()
cats = [i.replace('-',' ') for i in cats if i != 'Date']
cpiDf['Date'] = pd.to_datetime(cpiDf['Date'])



fin = pd.DataFrame()
for year in np.arange(2005,2022+1):
    spendDf = pd.read_csv(Path(f"resources/spending_{year}.csv"),index_col = 'Item')
    spendDf.drop(columns='Year',inplace=True,errors='ignore')
    spendDf = spendDf.loc[cats].T
    flatDf = {}
    for city in spendDf.index:
        for cat in spendDf.columns:
            flatDf[f'{cat} {city}'] = spendDf.loc[city,cat]
    flatDf = pd.DataFrame([flatDf])
    flatDf['Year'] = year

    fin = pd.concat([fin,flatDf],axis=0,ignore_index=True)

fin.dropna(axis='columns',inplace=True)
fin.set_index('Year',inplace=True)


grpCpi = cpiDf.groupby(cpiDf.Date.dt.year).mean()


yrAvgDf = pd.merge(fin,grpCpi,left_index=True,right_index=True,how='left')
yrAvgDf.reset_index(inplace=True)
yrAvgDf.head()

,Year,Cereals and bakery products Chicago,"Meats, poultry, fish, and eggs Chicago",Dairy products Chicago,Fruits and vegetables Chicago,Other food at home Chicago,Cereals and bakery products Detroit,"Meats, poultry, fish, and eggs Detroit",Dairy products Detroit,Fruits and vegetables Detroit,...,Cereals and bakery products Phoenix,"Meats, poultry, fish, and eggs Phoenix",Dairy products Phoenix,Fruits and vegetables Phoenix,Other food at home Phoenix,Cereals-and-bakery-products,"Meats,-poultry,-fish,-and-eggs",Dairy-products,Fruits-and-vegetables,Other-food-at-home
0,2005,473,815,365,609,1192,545,1047,405,635,...,470,859,415,632,1222,208.950000,184.741667,182.383333,241.391667,166.958333
1,2006,491,879,381,664,1315,527,931,416,638,...,482,894,433,672,1342,212.841667,186.550000,181.358333,252.925000,169.633333
2,2007,537,932,406,713,1457,579,871,420,635,...,508,873,445,676,1542,222.106917,195.616333,194.769500,262.628000,173.275167
3,2008,561,919,419,724,1476,628,868,456,677,...,496,741,406,607,1465,244.853167,204.653333,210.396000,278.932417,184.165667
4,2009,584,960,434,766,1506,540,849,435,670,...,464,800,362,608,1363,252.567083,203.804750,197.012500,272.944833,191.219667
